In [1]:
import csv
import numpy as np
import random

# A = np.random.randint(low=0,high=4095,size=(5,1), dtype=np.int64)
# with open('PWM_test.csv', 'w', newline='') as file:
#     mywriter = csv.writer(file, delimiter=',')
#     mywriter.writerows(A)
# input pwm value(initialise)

#can use this to add aberration or add it on pi,see pi_control.py file

In [2]:
import pygad
import numpy
import time
from pylablib.devices import Thorlabs

## initialise camera
Thorlabs.list_cameras_tlcam()
cam = Thorlabs.ThorlabsTLCamera(serial="14309")
cam.set_exposure(0.1)       # set exposure to 100ms
cam.set_roi(0,600,0,600)    # set 600x600px ROI in the upper left corner

def fitness_func(solution, solution_idx):
    # Calculating the fitness value of each solution in the current population.
    # solution is the PWM value for each channel
    solution = np.trunc(solution).astype(int)
    A = np.array([[solution[0]],[solution[1]],[solution[2]],[solution[3]],[solution[4]]])
    with open('PWM_test.csv', 'w', newline='') as file:
        mywriter = csv.writer(file, delimiter=',')
        mywriter.writerows(A)
    # wait
    time.sleep(0.1)    
    # take images
    cam.set_roi(0,600,0,600) # set 128x128px ROI in the upper left corner
    images = cam.grab(10) # grab 10 frames
    quality_number = get_quality_number(images) #quality number analysis method can be seen in other file.
    fitness = quality_number
    return fitness              # goal is letting fitness become biger and biger,fitness=quality number

fitness_function = fitness_func

num_generations = 10 # Number of generations.
num_parents_mating = 7 # Number of solutions to be selected as parents in the mating pool.

# To prepare the initial population, there are 2 ways:
# 1) Prepare it myself and pass it to the initial_population parameter. This way is useful when the user wants to start the genetic algorithm with a custom initial population.
# my custom initial population here can include: active zeros, defocus,cylindrical, spherical,etc.
# 2) Assign valid integer values to the sol_per_pop and num_genes parameters. If the initial_population parameter exists, then the sol_per_pop and num_genes parameters are useless.
sol_per_pop = 50 # Number of solutions in the population.
num_genes = 5
mutation_percent_genes = 1
init_range_low = 0         
init_range_high = 4095

last_fitness = 0
def callback_generation(ga_instance):
    global last_fitness
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution()[1]))
    print("Change     = {change}".format(change=ga_instance.best_solution()[1] - last_fitness))
    last_fitness = ga_instance.best_solution()[1]

Creating an instance of the GA class inside the ga module. Some parameters are initialized within the constructor.
ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating, 
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop, 
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       mutation_percent_genes=mutation_percent_genes,
                       on_generation=callback_generation)

# Running the GA to optimize the parameters of the function.
ga_instance.run()

# After the generations complete, some plots are showed that summarize the how the outputs/fitenss values evolve over generations.
ga_instance.plot_fitness()

# Returning the details of the best solution.
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Parameters of the best solution : {solution}".format(solution=solution))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))
print("Index of the best solution : {solution_idx}".format(solution_idx=solution_idx))

# prediction = numpy.sum(numpy.array(function_inputs)*solution)
# print("Predicted output based on the best solution : {prediction}".format(prediction=prediction))

if ga_instance.best_solution_generation != -1:
    print("Best fitness value reached after {best_solution_generation} generations.".format(best_solution_generation=ga_instance.best_solution_generation))

# Saving the GA instance.
filename = 'genetic' # The filename to which the instance is saved. The name is without extension.
ga_instance.save(filename=filename)

# Loading the saved GA instance.
loaded_ga_instance = pygad.load(filename=filename)
loaded_ga_instance.plot_fitness()

In [3]:
## close camera and are ready to run again
cam.close()